In [1]:
import findspark
findspark.init('/home/sdmohant/spark-3.0.0-bin-hadoop2.7')

In [2]:
import pyspark
from pyspark import SparkContext,SparkConf

In [3]:
from pyspark.sql import SparkSession

In [4]:
conf = SparkConf().set("spark.cores.max", "16") \
    .set("spark.driver.memory", "16g") \
    .set("spark.executor.memory", "16g") \
    .set("spark.executor.memory_overhead", "16g") \
    .set("spark.driver.maxResultsSize", "0") \
    .set("spark.sql.shuffle.partitions","300") \
    .set("spark.sql.execution.arrow.enabled","true") 

In [5]:
sc = SparkContext(appName="AllPapersAndFOS", conf=conf)

/home/sdmohant/spark-3.0.0-bin-hadoop2.7/python/pyspark/context.py:220: DeprecationWarning: Support for Python 2 and Python 3 prior to version 3.6 is deprecated as of Spark 3.0. See also the plan for dropping Python 2 support at https://spark.apache.org/news/plan-for-dropping-python-2-support.html.
  DeprecationWarning)


In [6]:
spark = SparkSession(sc)

In [7]:
from pyspark.sql.functions import isnan, count, col

In [8]:
df1= spark.read.options(header=True, inferSchema=True, delimiter=',').csv("PaperIDFOS.csv")

In [9]:
df1.show(2)

+-------+-------+--------------------+--------------------+---------------+-------------+----------------+----------+
| FOS_ID|FOSRank|   FOSNormalizedName|      FOSDisplayName|FOSDisplayLevel|FOSPaperCount|FOSCitationCount|   PaperID|
+-------+-------+--------------------+--------------------+---------------+-------------+----------------+----------+
|3910719|  13240|isotopes of vanadium|Isotopes of vanadium|              3|           72|             381|1963927026|
|3910719|  13240|isotopes of vanadium|Isotopes of vanadium|              3|           72|             381|2523956911|
+-------+-------+--------------------+--------------------+---------------+-------------+----------------+----------+
only showing top 2 rows



In [10]:
df1.count()

796470949

In [11]:
pap_df=spark.read.options(header=True, inferSchema=True, delimiter=',').csv("NewPaper.csv")

In [12]:
pap_df.count()

171227624

In [13]:
df1.select("PaperID").distinct().count()

145939476

In [14]:
pap_df.select("PaperID").distinct().count()

171227624

In [15]:
txt_file = sc.textFile("../Data/graph/2018-02-02/PaperFieldsOfStudy.txt")

In [16]:
temp_var = txt_file.map(lambda k: k.split("\t"))

In [17]:
paperFOS_DF = temp_var.toDF()

In [18]:
paperFOS_DF.show(2)

+----------+---------+-------------------+
|        _1|       _2|                 _3|
+----------+---------+-------------------+
|2514067917|204223013|0.43515353105522547|
|2514067917|530198007| 0.4671860155535934|
+----------+---------+-------------------+
only showing top 2 rows



In [19]:
from pyspark.sql.types import *

In [20]:
paperFOS_DF = paperFOS_DF.withColumn("PaperID",paperFOS_DF["_1"].cast(LongType()))

In [21]:
paperFOS_DF = paperFOS_DF.withColumn("FOS_ID",paperFOS_DF["_2"].cast(LongType()))

In [22]:
paperFOS_DF = paperFOS_DF.withColumn("FOSProbability",paperFOS_DF["_3"].cast(DoubleType()))

In [23]:
paperFOS_DF.show(2)

+----------+---------+-------------------+----------+---------+------------------+
|        _1|       _2|                 _3|   PaperID|   FOS_ID|    FOSProbability|
+----------+---------+-------------------+----------+---------+------------------+
|2514067917|204223013|0.43515353105522547|2514067917|204223013|0.4351535310552255|
|2514067917|530198007| 0.4671860155535934|2514067917|530198007|0.4671860155535934|
+----------+---------+-------------------+----------+---------+------------------+
only showing top 2 rows



In [24]:
drop_cols=["_1","_2","_3"]
paperFOS_DF = paperFOS_DF.drop(*drop_cols)

In [25]:
paperFOS_DF.count()

796470949

In [26]:
paperFOS_DF.coalesce(1).write.format('com.databricks.spark.csv').mode("overwrite").save('PaperFOSRaw.csv',header = 'true')

In [27]:
#paperFOS_DF.toPandas()

In [28]:
pap_fos_all = pap_df.join(df1, on=["PaperID"],how="inner")

In [29]:
pap_fos_all.show()

+-------+-----+--------------------+-------+--------------------+---------+----------+--------------------+---------+------------+--------------+------+-----+---------+--------+--------------+-------------+--------------+----------+-------+--------------------+--------------------+---------------+-------------+----------------+
|PaperID| Rank|                 DOI|DocType|          PaperTitle|PaperYear| PaperDate|           Publisher|JournalID|ConfSeriesID|ConfInstanceID|Volume|Issue|FirstPage|LastPage|ReferenceCount|CitationCount|EstimatedCount|    FOS_ID|FOSRank|   FOSNormalizedName|      FOSDisplayName|FOSDisplayLevel|FOSPaperCount|FOSCitationCount|
+-------+-----+--------------------+-------+--------------------+---------+----------+--------------------+---------+------------+--------------+------+-----+---------+--------+--------------+-------------+--------------+----------+-------+--------------------+--------------------+---------------+-------------+----------------+
|  17971|2

In [33]:
pap_fos_all.count()

796470949

In [34]:
pap_fos_all.columns

['PaperID',
 'Rank',
 'DocType',
 'PaperYear',
 'Publisher',
 'JournalID',
 'ConfSeriesID',
 'ConfInstanceID',
 'Volume',
 'Issue',
 'FirstPage',
 'LastPage',
 'ReferenceCount',
 'CitationCount',
 'EstimatedCount',
 'FOS_ID',
 'FOSRank',
 'FOSDisplayName',
 'FOSDisplayLevel',
 'FOSPaperCount',
 'FOSCitationCount']

In [31]:
drop_cols = ["PaperTitle","OriginalTitle","DOI","PaperDate","FOSNormalizedName"]
pap_fos_all = pap_fos_all.drop(*drop_cols)

In [32]:
from pyspark.sql.functions import isnan, when, count, col

In [33]:
pap_fos_all.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c) for c in pap_fos_all.columns]).show()

+-------+----+---------+---------+---------+---------+------------+--------------+---------+---------+---------+---------+--------------+-------------+--------------+------+-------+--------------+---------------+-------------+----------------+
|PaperID|Rank|  DocType|PaperYear|Publisher|JournalID|ConfSeriesID|ConfInstanceID|   Volume|    Issue|FirstPage| LastPage|ReferenceCount|CitationCount|EstimatedCount|FOS_ID|FOSRank|FOSDisplayName|FOSDisplayLevel|FOSPaperCount|FOSCitationCount|
+-------+----+---------+---------+---------+---------+------------+--------------+---------+---------+---------+---------+--------------+-------------+--------------+------+-------+--------------+---------------+-------------+----------------+
|      0|   0|354469500|        0|305794164|336350691|   766610545|     787098499|345558429|400003731|318183119|371285248|             0|            0|             0|     0|      0|            58|              0|            0|               0|
+-------+----+---------+

In [35]:
pap_fos_all.select("Volume","Issue","FirstPage","LastPage").show()

+------+-----+---------+--------+
|Volume|Issue|FirstPage|LastPage|
+------+-----+---------+--------+
|  null| null|     null|    null|
|  null| null|     null|    null|
|  null| null|     null|    null|
|  null| null|     null|    null|
|  null| null|     null|    null|
|  null| null|     null|    null|
|  null| null|     null|    null|
|  null| null|     null|    null|
|  null| null|     null|    null|
|  null| null|     null|    null|
|  null| null|      106|     111|
|  null| null|      106|     111|
|  null| null|      106|     111|
|  null| null|      106|     111|
|  null| null|     null|    null|
|  null| null|     null|    null|
|  null| null|     null|    null|
|    97|    3|      157|     168|
|    97|    3|      157|     168|
|    97|    3|      157|     168|
+------+-----+---------+--------+
only showing top 20 rows



In [36]:
pap_fos_all = pap_fos_all.withColumn("PaperAge", 2020-pap_fos_all["PaperYear"])

In [37]:
from pyspark.sql import functions as F

In [38]:
pap_fos_all = pap_fos_all.withColumn("IsJournal", F.when(F.col("DocType").isNotNull(), 0).otherwise(1))

In [39]:
pap_fos_all = pap_fos_all.withColumn("Volume", F.when(F.col("Volume").isNull(), 0).otherwise(F.col("Volume")))

In [40]:
pap_fos_all = pap_fos_all.withColumn("Issue", F.when(F.col("Issue").isNull(), 0).otherwise(F.col("Issue")))

In [41]:
pap_fos_all.select("Volume","Issue","PaperAge","IsJournal").show()

+------+-----+--------+---------+
|Volume|Issue|PaperAge|IsJournal|
+------+-----+--------+---------+
|     0|    0|      28|        0|
|     0|    0|      28|        0|
|     0|    0|      28|        0|
|     0|    0|      28|        0|
|     0|    0|      28|        0|
|     0|    0|      43|        1|
|     0|    0|      43|        1|
|     0|    0|      43|        1|
|     0|    0|      43|        1|
|     0|    0|      43|        1|
|     0|    0|       7|        1|
|     0|    0|       7|        1|
|     0|    0|       7|        1|
|     0|    0|       7|        1|
|     0|    0|      19|        1|
|     0|    0|      19|        1|
|     0|    0|      19|        1|
|    97|    3|      17|        1|
|    97|    3|      17|        1|
|    97|    3|      17|        1|
+------+-----+--------+---------+
only showing top 20 rows



In [48]:
pap_fos_all.select("PaperID","FOS_ID","FOSRank","FOSPaperCount","FOSCitationCount").show(40)

+-------+----------+-------+-------------+----------------+
|PaperID|    FOS_ID|FOSRank|FOSPaperCount|FOSCitationCount|
+-------+----------+-------+-------------+----------------+
|  17971| 144444463|  12363|        16525|           47645|
|  17971| 199639397|  10131|      1941652|         6118177|
|  17971| 127413603|  25000|     15585220|        44877184|
|  17971|  24326235|  11697|      4665517|        22343036|
|  17971|2779201687|  14423|         3758|            8799|
|  27933|  99454951|   9797|       396736|         2977821|
|  27933|  50660011|   9059|        26744|          413156|
|  27933|  71924100|   9323|     24318797|       212171917|
|  27933| 526734887|   9515|       475240|         1842827|
|  27933|  18918823|  10291|       195803|          826801|
|  29841|  96627749|  14794|         7943|             878|
|  29841|  23123220|   8378|       325725|         2390926|
|  29841|2776321320|   9551|        28458|          486141|
|  29841|  41008148|   7843|      990216

In [42]:
from pyspark.sql import Window
import pyspark.sql.functions as f
w = Window.partitionBy('PaperID')
temp_DF = pap_fos_all.select('PaperID', "FOSRank","FOSPaperCount","FOSCitationCount", f.min('FOSRank').over(w).alias('min_FOSRank')).sort('PaperID')

In [43]:
temp_DF.show(5)

+-------+-------+-------------+----------------+-----------+
|PaperID|FOSRank|FOSPaperCount|FOSCitationCount|min_FOSRank|
+-------+-------+-------------+----------------+-----------+
|     15|  10861|        16716|          104384|      10861|
|     15|  25000|      3362436|        10493865|      10861|
|     15|  10909|        74745|          121151|      10861|
|     23|  11565|      1371960|        11362571|       8720|
|     23|  10970|         9086|          147626|       8720|
+-------+-------+-------------+----------------+-----------+
only showing top 5 rows



In [44]:
temp_DF = temp_DF.select('PaperID',  "FOSRank","FOSPaperCount","FOSCitationCount","min_FOSRank", f.max('FOSRank').over(w).alias('max_FOSRank')).sort('PaperID', 'FOSRank')

In [45]:
#temp_DF = temp_DF.select('PaperID', 'FOSRank',"min_FOSRank", "max_FOSRank", f.max('FOSRank').over(w).alias('max_FOSRank')).sort('PaperID', 'FOSRank')

In [46]:
temp_DF = temp_DF.select('PaperID', 'FOSRank',"FOSPaperCount","FOSCitationCount", "min_FOSRank", "max_FOSRank", f.max('FOSPaperCount').over(w).alias('max_FOSPaperCount')).sort('PaperID', 'FOSPaperCount')

In [47]:
temp_DF = temp_DF.select('PaperID', 'FOSRank',"FOSPaperCount","FOSCitationCount","min_FOSRank", "max_FOSRank","max_FOSPaperCount", f.min('FOSPaperCount').over(w).alias('min_FOSPaperCount')).sort('PaperID')

In [48]:
temp_DF = temp_DF.select('PaperID', 'FOSRank',"FOSPaperCount","FOSCitationCount","min_FOSRank", "max_FOSRank","max_FOSPaperCount","min_FOSPaperCount", f.min('FOSCitationCount').over(w).alias('min_FOSCitationCount')).sort('PaperID', 'FOSCitationCount')

In [49]:
temp_DF = temp_DF.select('PaperID', 'FOSRank',"FOSPaperCount","FOSCitationCount","min_FOSRank", "max_FOSRank","max_FOSPaperCount","min_FOSPaperCount","min_FOSCitationCount", f.max('FOSCitationCount').over(w).alias('max_FOSCitationCount')).sort('PaperID', 'FOSCitationCount')

In [50]:
temp_DF.show(5)

+-------+-------+-------------+----------------+-----------+-----------+-----------------+-----------------+--------------------+--------------------+
|PaperID|FOSRank|FOSPaperCount|FOSCitationCount|min_FOSRank|max_FOSRank|max_FOSPaperCount|min_FOSPaperCount|min_FOSCitationCount|max_FOSCitationCount|
+-------+-------+-------------+----------------+-----------+-----------+-----------------+-----------------+--------------------+--------------------+
|     15|  10861|        16716|          104384|      10861|      25000|          3362436|            16716|              104384|            10493865|
|     15|  10909|        74745|          121151|      10861|      25000|          3362436|            16716|              104384|            10493865|
|     15|  25000|      3362436|        10493865|      10861|      25000|          3362436|            16716|              104384|            10493865|
|     23|  10970|         9086|          147626|       8720|      11565|         24318797|    

In [51]:
temp_DF1 = temp_DF.groupBy("PaperID").mean()

In [72]:
temp_DF1.show(10)

+-------+------------+------------------+------------------+---------------------+----------------+----------------+----------------------+----------------------+-------------------------+-------------------------+
|PaperID|avg(PaperID)|      avg(FOSRank)|avg(FOSPaperCount)|avg(FOSCitationCount)|avg(min_FOSRank)|avg(max_FOSRank)|avg(max_FOSPaperCount)|avg(min_FOSPaperCount)|avg(min_FOSCitationCount)|avg(max_FOSCitationCount)|
+-------+------------+------------------+------------------+---------------------+----------------+----------------+----------------------+----------------------+-------------------------+-------------------------+
|     15|        15.0|           15590.0|         1151299.0|   3573133.3333333335|         10861.0|         25000.0|             3362436.0|               16716.0|                 104384.0|              1.0493865E7|
|     23|        23.0|           10144.5|        7424361.25|        6.706911625E7|          8720.0|         11565.0|           2.4318797E7| 

In [52]:
drop_cols = ["FOSCitationCount","FOSPaperCount","FOSDisplayLevel","FOSDisplayName","FOSRank"]
pap_fos_all = pap_fos_all.drop(*drop_cols)


In [74]:
pap_fos_all.show(10)

+-------+-----+-------+---------+---------+---------+------------+--------------+------+-----+---------+--------+--------------+-------------+--------------+----------+--------+---------+
|PaperID| Rank|DocType|PaperYear|Publisher|JournalID|ConfSeriesID|ConfInstanceID|Volume|Issue|FirstPage|LastPage|ReferenceCount|CitationCount|EstimatedCount|    FOS_ID|PaperAge|IsJournal|
+-------+-----+-------+---------+---------+---------+------------+--------------+------+-----+---------+--------+--------------+-------------+--------------+----------+--------+---------+
|  17971|23209| Patent|     1992|     null|     null|        null|          null|     0|    0|     null|    null|             0|            2|             2| 144444463|      28|        0|
|  17971|23209| Patent|     1992|     null|     null|        null|          null|     0|    0|     null|    null|             0|            2|             2| 199639397|      28|        0|
|  17971|23209| Patent|     1992|     null|     null|       

In [53]:
pap_fos_all2 = pap_fos_all.drop_duplicates(subset=['PaperID'])

In [77]:
pap_fos_all2.show(10)

+-------+-----+-------+---------+----------------------+---------+------------+--------------+------+-----+---------+--------+--------------+-------------+--------------+---------+--------+---------+
|PaperID| Rank|DocType|PaperYear|             Publisher|JournalID|ConfSeriesID|ConfInstanceID|Volume|Issue|FirstPage|LastPage|ReferenceCount|CitationCount|EstimatedCount|   FOS_ID|PaperAge|IsJournal|
+-------+-----+-------+---------+----------------------+---------+------------+--------------+------+-----+---------+--------+--------------+-------------+--------------+---------+--------+---------+
|  17971|23209| Patent|     1992|                  null|     null|        null|          null|     0|    0|     null|    null|             0|            2|             2|144444463|      28|        0|
|  27933|26437|   null|     1977|                  null| 92085786|        null|          null|     0|    0|     null|    null|             0|            0|             0| 99454951|      43|        1|


In [78]:
pap_fos_all2.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c) for c in pap_fos_all.columns]).show()

+-------+----+--------+---------+---------+---------+------------+--------------+------+-----+---------+--------+--------------+-------------+--------------+------+--------+---------+
|PaperID|Rank| DocType|PaperYear|Publisher|JournalID|ConfSeriesID|ConfInstanceID|Volume|Issue|FirstPage|LastPage|ReferenceCount|CitationCount|EstimatedCount|FOS_ID|PaperAge|IsJournal|
+-------+----+--------+---------+---------+---------+------------+--------------+------+-----+---------+--------+--------------+-------------+--------------+------+--------+---------+
|      0|   0|73691848|        0| 60446008| 68832819|   141885753|     144685640|     0|    0| 62701649|73931016|             0|            0|             0|     0|       0|        0|
+-------+----+--------+---------+---------+---------+------------+--------------+------+-----+---------+--------+--------------+-------------+--------------+------+--------+---------+



In [54]:
drop_cols = ["FirstPage","LastPage","DocType"]
pap_fos_all2 = pap_fos_all2.drop(*drop_cols)


In [55]:
temp_DF_new  = temp_DF.drop_duplicates(subset=['PaperID'])

In [82]:
temp_DF_new.show(2)

+-------+-------+-------------+----------------+-----------+-----------+-----------------+-----------------+--------------------+--------------------+
|PaperID|FOSRank|FOSPaperCount|FOSCitationCount|min_FOSRank|max_FOSRank|max_FOSPaperCount|min_FOSPaperCount|min_FOSCitationCount|max_FOSCitationCount|
+-------+-------+-------------+----------------+-----------+-----------+-----------------+-----------------+--------------------+--------------------+
|     15|  10861|        16716|          104384|      10861|      25000|          3362436|            16716|              104384|            10493865|
|     23|  10970|         9086|          147626|       8720|      11565|         24318797|             9086|              147626|           212171917|
+-------+-------+-------------+----------------+-----------+-----------+-----------------+-----------------+--------------------+--------------------+
only showing top 2 rows



In [56]:
drop_cols = ["FOSRank","FOSPaperCount","FOSCitationCount"]
temp_DF_new = temp_DF_new.drop(*drop_cols)


In [57]:
temp_DF_new.count()

145939476

In [58]:
pap_fos_all2.count()

145939476

In [59]:
pap_fos_all3 = pap_fos_all2.join(temp_DF_new,on=["PaperID"], how="inner")

In [60]:
pap_fos_all3.show(2)

+-------+-----+---------+---------+---------+------------+--------------+------+-----+--------------+-------------+--------------+---------+--------+---------+-----------+-----------+-----------------+-----------------+--------------------+--------------------+
|PaperID| Rank|PaperYear|Publisher|JournalID|ConfSeriesID|ConfInstanceID|Volume|Issue|ReferenceCount|CitationCount|EstimatedCount|   FOS_ID|PaperAge|IsJournal|min_FOSRank|max_FOSRank|max_FOSPaperCount|min_FOSPaperCount|min_FOSCitationCount|max_FOSCitationCount|
+-------+-----+---------+---------+---------+------------+--------------+------+-----+--------------+-------------+--------------+---------+--------+---------+-----------+-----------+-----------------+-----------------+--------------------+--------------------+
|  17971|23209|     1992|     null|     null|        null|          null|     0|    0|             0|            2|             2|144444463|      28|        0|      10131|      25000|         15585220|             

In [61]:
papref = spark.read.options(header = True, inferSchema = True,delimiter=',').csv("PaperReferences.csv")

In [62]:
papref.show(2)

+----------+-----+----------+-----------+
|        _1|count|   PaperID|PaperRefCnt|
+----------+-----+----------+-----------+
|2252294392|    8|2252294392|          8|
|1479838528|   79|1479838528|         79|
+----------+-----+----------+-----------+
only showing top 2 rows



In [63]:
pap_fos_all3.count()

145939476

In [64]:
drop_cols = ["_1","count"]
papref = papref.drop(*drop_cols)

In [65]:
pap_fos_all4 = pap_fos_all3.join(papref,on=["PaperID"], how="left")

In [66]:
pap_fos_all4.count()

145939476

In [67]:
pap_fos_all4 = pap_fos_all4.withColumn("PaperRefCnt", F.when(F.col("PaperRefCnt").isNull(), 0).otherwise(F.col("Issue")))

In [98]:
pap_fos_all4.show(10)

+-------+-----+---------+----------------------+---------+------------+--------------+------+-----+--------------+-------------+--------------+---------+--------+---------+-----------+-----------+-----------------+-----------------+--------------------+--------------------+-----------+
|PaperID| Rank|PaperYear|             Publisher|JournalID|ConfSeriesID|ConfInstanceID|Volume|Issue|ReferenceCount|CitationCount|EstimatedCount|   FOS_ID|PaperAge|IsJournal|min_FOSRank|max_FOSRank|max_FOSPaperCount|min_FOSPaperCount|min_FOSCitationCount|max_FOSCitationCount|PaperRefCnt|
+-------+-----+---------+----------------------+---------+------------+--------------+------+-----+--------------+-------------+--------------+---------+--------+---------+-----------+-----------+-----------------+-----------------+--------------------+--------------------+-----------+
|  17971|23209|     1992|                  null|     null|        null|          null|     0|    0|             0|            2|           

In [68]:
drop_cols = ["Publisher","JournalID","ConfSeriesID","ConfInstanceID"]
pap_fos_all5 = pap_fos_all4.drop(*drop_cols)

In [69]:
pap_fos_all5.coalesce(1).write.format('com.databricks.spark.csv').mode("overwrite").save('AllPapersFOS1.csv',header = 'true')

In [71]:
papauthaff= spark.read.options(header=True, inferSchema=True, delimiter=',').csv("PaperAuthorAffilitions.csv")

In [72]:
papauthaff.show(2)

+----------+----------+-------------+--------------------+
|   PaperID|  AuthorId|AffiliationId|AuthorSequenceNumber|
+----------+----------+-------------+--------------------+
|2514067917|2165266595|   1334877674|                   1|
|2514067917|2334513227|   1334877674|                   2|
+----------+----------+-------------+--------------------+
only showing top 2 rows



In [73]:
pap_fos_auth = pap_fos_all4.join(papauthaff,on="PaperID",how="right")

In [74]:
pap_fos_auth.count()

223953283

In [75]:
pap_fos_auth.show(10)

+-------+-----+---------+--------------------+---------+------------+--------------+------+-----+--------------+-------------+--------------+---------+--------+---------+-----------+-----------+-----------------+-----------------+--------------------+--------------------+-----------+----------+-------------+--------------------+
|PaperID| Rank|PaperYear|           Publisher|JournalID|ConfSeriesID|ConfInstanceID|Volume|Issue|ReferenceCount|CitationCount|EstimatedCount|   FOS_ID|PaperAge|IsJournal|min_FOSRank|max_FOSRank|max_FOSPaperCount|min_FOSPaperCount|min_FOSCitationCount|max_FOSCitationCount|PaperRefCnt|  AuthorId|AffiliationId|AuthorSequenceNumber|
+-------+-----+---------+--------------------+---------+------------+--------------+------+-----+--------------+-------------+--------------+---------+--------+---------+-----------+-----------+-----------------+-----------------+--------------------+--------------------+-----------+----------+-------------+--------------------+
|  2793

In [78]:
pap_fos_auth.select(["PaperID","AuthorId","AffiliationId","FOS_ID"]).show(15)

+-------+----------+-------------+---------+
|PaperID|  AuthorId|AffiliationId|   FOS_ID|
+-------+----------+-------------+---------+
|  27933|1229029628|    202963720| 99454951|
|  27933|2004716990|    202697423| 99454951|
|  27933|2172756124|    202697423| 99454951|
|  29841|2123350797|     99542240| 96627749|
|  66606|2175418804|    141945490| 75630572|
|  66606|1119406721|     86519309| 75630572|
|  66606|2616645929|    103163165| 75630572|
|  66606|2569699948|    200719446| 75630572|
|  71530|2153977648|     13134134| 71924100|
|  74185|2164071203|    106157433|144024400|
| 111538|2018993203|   1313323035|127385683|
| 111538|2029252525|     19880235|127385683|
| 156454|2731245360|   1306287861|162269090|
| 199209|2169118548|   1294824412|185592680|
| 199209| 285337606|    130828816|185592680|
+-------+----------+-------------+---------+
only showing top 15 rows



In [79]:
aff = spark.read.options(header=True, inferSchema=True, delimiter=',').csv("Affiliations.csv")

In [80]:
aff.show(2)

+-------------+---------------+--------------------+--------------------+------------+---------------+
|AffiliationId|AffiliationRank|    AfNormalizedName|       AfDisplayName|AfPaperCount|AfCitationCount|
+-------------+---------------+--------------------+--------------------+------------+---------------+
|    154099455|           8223| shandong university| Shandong University|       86381|        2184963|
|     94975175|          10705|des moines univer...|Des Moines Univer...|        1197|          25564|
+-------------+---------------+--------------------+--------------------+------------+---------------+
only showing top 2 rows



In [84]:
pap_fos_au_aff = pap_fos_auth.join(aff,on="AffiliationId",how="inner")

In [85]:
pap_fos_au_aff.select(["PaperID","AuthorId","AffiliationId","FOS_ID"]).show(10)

+-------+----------+-------------+---------+
|PaperID|  AuthorId|AffiliationId|   FOS_ID|
+-------+----------+-------------+---------+
|  27933|1229029628|    202963720| 99454951|
|  27933|2004716990|    202697423| 99454951|
|  27933|2172756124|    202697423| 99454951|
|  29841|2123350797|     99542240| 96627749|
|  66606|2175418804|    141945490| 75630572|
|  66606|1119406721|     86519309| 75630572|
|  66606|2616645929|    103163165| 75630572|
|  66606|2569699948|    200719446| 75630572|
|  71530|2153977648|     13134134| 71924100|
|  74185|2164071203|    106157433|144024400|
+-------+----------+-------------+---------+
only showing top 10 rows



In [86]:
auth = spark.read.options(header=True, inferSchema=True, delimiter=',').csv("Author.csv")

In [88]:
pap_fos_au_aff_all = pap_fos_au_aff.join(auth,on="AuthorId",how="inner")

In [89]:
pap_fos_au_aff_all.select(["PaperID","AuthorId","AffiliationId","FOS_ID"]).show(10)

+----------+--------+-------------+----------+
|   PaperID|AuthorId|AffiliationId|    FOS_ID|
+----------+--------+-------------+----------+
|2129465055|   15437|     32389192|2780149156|
|2148676614|   15437|     32389192|2776888751|
|2324684188|   15437|     32389192|  71924100|
|2088556209|   15437|     32389192|2778742482|
|2175552737|   15437|     32389192|2776858399|
|1994839481|   15437|     32389192|2780149156|
|2765447610|   15437|     32389192| 143753070|
|2191263223|   15437|     32389192|2776858399|
|1976958153|   15437|     32389192|2780605848|
|2095223300|   15437|     32389192|  71924100|
+----------+--------+-------------+----------+
only showing top 10 rows



In [90]:
pap_fos_au_aff_all.sort(["PaperID"]).select(["PaperID","AuthorId","AffiliationId","FOS_ID"]).show(15)

+-------+----------+-------------+----------+
|PaperID|  AuthorId|AffiliationId|    FOS_ID|
+-------+----------+-------------+----------+
|     23|1243978490|     79576946|2778602436|
|     37|2767506227|     30338065|2779982284|
|     37|2151921381|     30338065|2779982284|
|    108|2126642415|    165102784| 124952713|
|    125|2250382311|    169199633| 201995342|
|    125|1995014452|    169199633| 201995342|
|    125|2002579779|    169199633| 201995342|
|    125|2283694448|    169199633| 201995342|
|    136|2192065579|    154570441|  75639521|
|    136|2397980907|    201448701|  75639521|
|    138|2726581387|   1316910977| 185592680|
|    138|2729948773|   1316910977| 185592680|
|    193|1234409970|     78577930| 124952713|
|    236|2102314242|     23732399|  29985473|
|    266|2119155047|     97018004| 159985019|
+-------+----------+-------------+----------+
only showing top 15 rows



In [91]:
pap_fos_au_aff.sort(["PaperID"]).select(["PaperID","AuthorId","AffiliationId","FOS_ID"]).show(15)

+-------+----------+-------------+----------+
|PaperID|  AuthorId|AffiliationId|    FOS_ID|
+-------+----------+-------------+----------+
|     23|1243978490|     79576946|2778602436|
|     37|2767506227|     30338065|2779982284|
|     37|2151921381|     30338065|2779982284|
|    108|2126642415|    165102784| 124952713|
|    125|2002579779|    169199633| 201995342|
|    125|1995014452|    169199633| 201995342|
|    125|2250382311|    169199633| 201995342|
|    125|2283694448|    169199633| 201995342|
|    136|2397980907|    201448701|  75639521|
|    136|2192065579|    154570441|  75639521|
|    138|2729948773|   1316910977| 185592680|
|    138|2726581387|   1316910977| 185592680|
|    193|1234409970|     78577930| 124952713|
|    236|2102314242|     23732399|  29985473|
|    266|2119155047|     97018004| 159985019|
+-------+----------+-------------+----------+
only showing top 15 rows



In [92]:
pap_fos_au_aff_all.columns

['AuthorId',
 'AffiliationId',
 'PaperID',
 'Rank',
 'PaperYear',
 'Publisher',
 'JournalID',
 'ConfSeriesID',
 'ConfInstanceID',
 'Volume',
 'Issue',
 'ReferenceCount',
 'CitationCount',
 'EstimatedCount',
 'FOS_ID',
 'PaperAge',
 'IsJournal',
 'min_FOSRank',
 'max_FOSRank',
 'max_FOSPaperCount',
 'min_FOSPaperCount',
 'min_FOSCitationCount',
 'max_FOSCitationCount',
 'PaperRefCnt',
 'AuthorSequenceNumber',
 'AffiliationRank',
 'AfNormalizedName',
 'AfDisplayName',
 'AfPaperCount',
 'AfCitationCount',
 'AuthorRank',
 'AutNormalizedName',
 'AutDisplayName',
 'AutLastAffiliation',
 'AutPaperCount',
 'AutCitationCount']

In [93]:
pap_fos_au_aff_all.count()

223953283

In [94]:
from pyspark.sql import Window
import pyspark.sql.functions as f
w = Window.partitionBy('PaperID')
temp_DF = pap_fos_au_aff_all.select('PaperID', 'AuthorRank', f.min('AuthorRank').over(w).alias('min_AuthorRank')).sort('PaperID')

In [96]:
temp_DF.show(4)

+-------+----------+--------------+
|PaperID|AuthorRank|min_AuthorRank|
+-------+----------+--------------+
|     23|     17530|         17530|
|     37|     19697|         19569|
|     37|     19569|         19569|
|    108|     19728|         19728|
+-------+----------+--------------+
only showing top 4 rows



In [ ]:
temp_DF1 = pap_fos_au_aff_all.select('PaperID', 'AuthorRank',"min_AuthorRank", f.max('AuthorRank').over(w).alias('max_AuthorRank')).sort('PaperID', 'AuthorRank')